In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
# import 
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
df = pd.read_excel(r'D:\MSc Software and AI\Team Project\TurbineTeamProject\data\dundalk.xlsx')

In [3]:
df.set_index(['Timestamps'], inplace= True)

In [4]:
high_corr_features = ['WindSpeed',
 'GenPh1Temp',
 'GenPh2Temp',
 'GenPh3Temp',
 'Power']

In [ ]:
# scaler_X = MinMaxScaler()
# scaler_y = MinMaxScaler()

In [ ]:
df_arima = df[high_corr_features]

In [16]:
minmax = MinMaxScaler()

In [20]:
df_arima.head()

,WindSpeed,GenPh1Temp,GenPh2Temp,GenPh3Temp,Power
Timestamps,,,,,
2006-01-30 20:30:00,9.5,82,82,81,527.3
2006-01-30 20:40:00,9.9,86,85,85,584.5
2006-01-30 20:50:00,9.3,89,88,88,527.3
2006-01-30 21:00:00,8.7,88,88,88,428.0
2006-01-30 21:10:00,8.6,85,85,84,434.7


In [21]:
df_arima_scaled = df_arima.copy()
df_arima_scaled[df_arima_scaled.columns] = minmax.fit_transform(df_arima)

In [22]:
df_arima_scaled.head()

,WindSpeed,GenPh1Temp,GenPh2Temp,GenPh3Temp,Power
Timestamps,,,,,
2006-01-30 20:30:00,0.025787,0.630769,0.401961,0.618321,0.591874
2006-01-30 20:40:00,0.026873,0.661538,0.416667,0.648855,0.600778
2006-01-30 20:50:00,0.025244,0.684615,0.431373,0.671756,0.591874
2006-01-30 21:00:00,0.023616,0.676923,0.431373,0.671756,0.576417
2006-01-30 21:10:00,0.023344,0.653846,0.416667,0.641221,0.577460


In [47]:
train_data = df_arima_scaled.loc[:'2018-09-10'] #04-10-2018
test_data = df_arima_scaled.loc['2018-09-11':'2018-10-02']

In [48]:
train_X, train_y = train_data.drop(['Power'], axis=1), train_data['Power']
test_X, test_y = test_data.drop(['Power'], axis=1), test_data['Power']

In [49]:
# Define SARIMAX model (ARIMA + wind speed as exogenous variable)
p=2
q=1
model = SARIMAX(train_y, exog=train_X, order=(p, 1, q))

# Fit model
model_fit = model.fit()

# Print model summary
print(model_fit.summary())


c:\Users\rsaxe\.conda\envs\yolov8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\rsaxe\.conda\envs\yolov8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:                  Power   No. Observations:               578549
Model:               SARIMAX(2, 1, 1)   Log Likelihood             2015969.797
Date:                Sun, 23 Feb 2025   AIC                       -4031923.595
Time:                        13:39:13   BIC                       -4031833.449
Sample:                             0   HQIC                      -4031898.229
                             - 578549                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
WindSpeed      1.7078   9.06e-05   1.88e+04      0.000       1.708       1.708
GenPh1Temp     0.1465      0.002     97.407      0.000       0.144       0.149
GenPh2Temp    -0.0937      0.000   -506.671      0.0

In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
# Use actual timestamps for start and end
start_date = train_y.index[0]  # First timestamp of test data
end_date = test_y.index[-1]   # Last timestamp of test data

# Make predictions using the actual timestamps
predictions = model_fit.predict(start=start_date, end=end_date, exog=test_X)

# Evaluate performance
mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
mpe = mean_absolute_percentage_error(test_y, predictions)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"mean_absolute_percentage_error: {mpe}")



KeyError: 'The `end` argument could not be matched to a location related to the index of the data.'

In [ ]:
predictions_df = pd.DataFrame({'timestamp': test_y.index, 'predicted_power': predictions})
predictions_df.set_index('timestamp', inplace=True)
print(predictions_df.head())

In [ ]:
plt.figure(figsize=(12,6))

# Plot actual test data
plt.plot(test_y.index, test_y, label="Actual Power Output", color="blue")

# Plot predicted data
plt.plot(predictions_df.index, predictions_df['predicted_power'], label="Predicted Power Output", 
         color="red", linestyle="dashed")

plt.xlabel("Date")
plt.ylabel("Power Output (kW)")
plt.title("ARIMAX Power Forecasting: March Training → April Prediction")
plt.legend()
plt.show()
